In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt
from datetime import datetime

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

fp = '../../SnowEx-Data'
lidar_dir = Path(fp)
fps = list(lidar_dir.glob('*.nc'))
fps = [f for f in fps if '.sub.nc' not in str(f)]
fps = [f for f in fps if '.old.nc' not in str(f)]

xs = []
ys = []

for fp in fps:
    ds = xr.open_dataset(fp)
    try:
        ft = pd.to_datetime(ds.attrs['lidar-flight-time'])
    except KeyError:

        ft = pd.to_datetime(fp.stem.split('_')[1].strip('.old'))

    dt = pd.Timedelta('3 days')
    loc = fp.stem.split('_')[0]
    if loc == 'Little':
        loc = 'Little Cottonwood'

    lidar_sd = ds['lidar-sd']
    
    xs.append(ds['lidar-sd'].values.ravel())
    ys.append(ds['snow_depth'].sel(time = slice(ft - dt, ft + dt)).where(~lidar_sd.isnull()).mean(dim = 'time').values.ravel())

# Frasier now
ds = xr.open_dataset('../../SnowEx-Data/Frasier_2021-03-19.old.nc')
xs.append(ds['lidar-sd'].isel(time= 56).values.ravel())
ys.append(ds['snow_depth'].isel(time = 59).where(~ds['lidar-sd'].isel(time= 56).isnull()).values.ravel())

ds = xr.open_dataset('/Users/zachkeskinen/Documents/spicy-snow/SnowEx-Data/Frasier_2020-02-11.old.nc')
xs.append(ds['lidar-sd'].values.ravel())
ys.append(ds['snow_depth'].isel(time = 48).where(~ds['lidar-sd'].isnull()).values.ravel())

# stack arrays
xs = np.hstack(xs)
ys = np.hstack(ys)

xs_tmp = xs[~np.isnan(xs) & ~np.isnan(ys) & (ys > 0)]
ys = ys[~np.isnan(xs) & ~np.isnan(ys) & (ys > 0)]
xs = xs_tmp

In [ ]:
import matplotlib as mpl
n = 1
plt.figure(figsize = (6, 6))
plt.hist2d(xs, ys*n, bins = 150, norm = mpl.colors.LogNorm())
plt.xlim(0, 5)
plt.ylim(0, 5)
plt.xlabel('Lidar Snow Depth')
plt.ylabel('Spicy Snow Depth')
plt.plot([0, 10], [0,10], color = 'black')

from sklearn.metrics import mean_squared_error
rms = mean_squared_error(xs, ys*n, squared=False)
from scipy.stats import pearsonr
r, p = pearsonr(xs, ys*n)

plt.text(.99, .01, f'RMSE: {rms:.2}, r: {r:.2}', ha='right', va='bottom', transform=plt.gca().transAxes)
plt.show()

In [ ]:
import matplotlib.patches as mpatches
labels = []
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))
    return violin

for i in range(4):
    sub_xs = xs[(xs < i + 1) & (xs > i)]
    sub_ys = ys[(xs < i + 1) & (xs > i)]
    if i == 0:
        v1 = add_label(plt.violinplot(sub_xs, vert= True, positions = [i + 0.5], showmeans=False, showextrema=False, showmedians=False), 'Lidar')
    else:
        v1 = plt.violinplot(sub_xs, vert= True, positions = [i + 0.5], showmeans=False, showextrema=False, showmedians=False)
    for b in v1['bodies']:
        # get the center
        m = np.mean(b.get_paths()[0].vertices[:, 0])
        # modify the paths to not go further right than the center
        b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], -np.inf, m)
        b.set_color('C0')
    
    if i == 0:
        v1 = add_label(plt.violinplot(sub_ys, vert= True, positions = [i + 0.5], showmeans=False, showextrema=False, showmedians=False), 'Spicy')
    else:    
        v1 = plt.violinplot(sub_ys, vert= True, positions = [i + 0.5], showmeans=False, showextrema=False, showmedians=False)
    for b in v1['bodies']:
        # get the center
        m = np.mean(b.get_paths()[0].vertices[:, 0])
        # modify the paths to not go further left than the center
        b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], m, np.inf)
        b.set_color('C1')

plt.xticks([0.5, 1.5, 2.5, 3.5], labels = ['0 - 1 m', '1 - 2 m','2 - 3 m','3 - 4 m'])
plt.ylabel('Snow Depth')
plt.legend(*zip(*labels), loc=2)
